In [ ]:
# /**********************************************************************************************************

# File Name		: 	customer_group.ipynb
# Purpose			:   customizing kmeans model output into bigquery table
# Author			:   DeepSphere.AI, Inc.
# Date and Time 	: 	03/16/2021 10:30 hrs
# Version			: 	1.0 

# /************************************************************************************************************


def read_op_data_bqml(group_num):
    import google.auth
    from google.cloud import bigquery
    from google.cloud import bigquery_storage

    # Explicitly create a credentials object. This allows you to use the same
    # credentials for both the BigQuery and BigQuery Storage clients, avoiding
    # unnecessary API calls to fetch duplicate authentication tokens.
    vAR_client = bigquery.Client()
    vAR_query_string = """SELECT customer_id,CENTROID_ID,created_time,platform_source,language,location,gender,age FROM `vast-verve-292018.ds_ai.ds_ai_kmeans_op` where CENTROID_ID="""
    vAR_query_string = vAR_query_string+group_num
    vAR_df = vAR_client.query(vAR_query_string).to_dataframe()
    
    return vAR_df

In [ ]:
def pf_source(df):
    vAR_df_source_list = []
    for lCount_idx in df.platform_source.unique():
        vAR_df_source_list.append(lCount_idx)
    return vAR_df_source_list

In [ ]:
def group_customer_data_bqml():
    vAR_df_0 = read_op_data_bqml("1")
    vAR_df_1 = read_op_data_bqml("2")
    vAR_df_2 = read_op_data_bqml("3")
    vAR_platform_source_list_0 = pf_source(vAR_df_0)
    vAR_platform_source_list_1 = pf_source(vAR_df_1)
    vAR_platform_source_list_2 = pf_source(vAR_df_2)
    for ps in vAR_platform_source_list_0:
        vAR_df_0[str(ps)] = 'yes'
    for ps in vAR_platform_source_list_1:
        vAR_df_1[str(ps)] = 'yes'
    for ps in vAR_platform_source_list_2:
        vAR_df_2[str(ps)] = 'yes'
    vAR_ds = vAR_df_0.append(vAR_df_1)
    vAR_ds_org = vAR_ds.append(vAR_df_2)
    return vAR_ds_org

In [ ]:
def bq_insert_bqml():
    import pandas as pd
    from google.cloud import bigquery

    # Example data
    vAR_df = group_customer_data_bqml()

    # Load client
    vAR_client = bigquery.Client(project='vast-verve-292018')

    # Define table name, in format dataset.table_name
    vAR_table = 'ds_ai.ds_ai_customised_kmeans_op'
    vAR_job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")
    # Load data to BQ
    vAR_job = vAR_client.load_table_from_dataframe(vAR_df, vAR_table,job_config=vAR_job_config)
    return True

In [ ]:
# select *,
# case when twitter = 'yes' and facebook='yes'
# then 'yes' else null end as twitter_facebook,
# case when twitter = 'yes' and youtube='yes'
# then 'yes' else null end as twitter_youtube,
# case when youtube = 'yes' and facebook='yes'
# then 'yes' else null end as youtube_facebook,
# case when twitter = 'yes' and facebook='yes' and youtube='yes'
# then 'yes' else null end as twitter_facebook_youtube,
# FROM `vast-verve-292018.ds_ai.ds_ai_customised_kmeans_op`

In [ ]:
# /**********************************************************************************************************
# Disclaimer.

# We are providing this code block strictly for learning and researching, this is not a production ready code.
# We have no liability on this particular code under any circumstances; users should
# use this code on their own risk. All software,hardware and other products that are
# referenced in these materials belong to the respective vendor who developed or who owns 
# this product.
# /**********************************************************************************************************
